In [0]:
#Проверяю качество работы модели на 1000 запросах

# Поиск с Fasttext

In [0]:
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors

In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#model_file = '/content/drive/My Drive/fasttext/model.model'

In [ ]:
model_file = 'fasttext/model.model'

In [0]:
model_fasttext = KeyedVectors.load(model_file)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
import csv
with open('quora_question_pairs_rus.csv', 'r', encoding = 'utf-8') as j:
    reader = csv.reader(j)
    file = list(reader)

In [0]:
docs = []
for indx, row in enumerate(file):
    if indx > 0:
        docs.append([row[2], row[3]])

In [0]:
!pip install pymorphy2

     |████████████████████████████████| 51kB 2.1MB/s 
     |████████████████████████████████| 7.1MB 8.3MB/s 


In [0]:
import pymorphy2 as pm2
pmm = pm2.MorphAnalyzer()
import re

In [0]:
import nltk
nltk.download("stopwords")

from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
corpus = []
for indx, row in enumerate(file):
    if indx > 0:
      
        doc = row[2]
        doc = re.sub(r'[^\w\s]', '', doc)
        doc = re.sub(r'\d', '', doc)
        
        doc = re.sub(r'[A-Za-z]', '', doc)
        doc_arr = [pmm.normal_forms(x)[0] for x in doc.split() if x not in russian_stopwords]
        doc = ' '.join(doc_arr)
        
        corpus.append(doc)

In [0]:
len(corpus)

404288

In [0]:
import numpy as np

In [0]:
%%time

matrix = []
for doc in corpus:
    doc = doc.split()
    lemmas_vectors = np.zeros((len(doc), model_fasttext.vector_size))
    vec = np.zeros((model_fasttext.vector_size,))
  
    for idx, lemma in enumerate(doc):
        if lemma in model_fasttext.vocab:
            lemmas_vectors[idx] = model_fasttext.wv[lemma]
        
    if lemmas_vectors.shape[0] is not 0:
        vec = np.mean(lemmas_vectors, axis=0)
  
    matrix.append(vec)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # Remove the CWD from sys.path while we load stuff.


CPU times: user 24.7 s, sys: 1.96 s, total: 26.7 s
Wall time: 26.8 s


In [0]:
len(matrix)

404288

In [0]:
np_matrix = np.asarray(matrix)
rev_matrix = np.transpose(np_matrix)

In [0]:
queries = []
for indx, row in enumerate(file):
    if indx > 0:
      
        query = row[1]
        query = re.sub(r'[^\w\s]', '', query)
        query = re.sub(r'\d', '', query)
        
        query = re.sub(r'[A-Za-z]', '', query)
        query_arr = [pmm.normal_forms(x)[0] for x in query.split() if x not in russian_stopwords]
        query = ' '.join(query_arr)
        
        queries.append(query)

In [0]:
len(queries)

404288

In [0]:
success = []

quers = queries[:1000]

for query in quers:

    q = query.split()
    lemmas_vectors = np.zeros((len(q), model_fasttext.vector_size))
    vec = np.zeros((model_fasttext.vector_size,))
  
    for idx, lemma in enumerate(q):
        if lemma in model_fasttext.vocab:
            lemmas_vectors[idx] = model_fasttext.wv[lemma]
        
    if lemmas_vectors.shape[0] is not 0:
        vec = np.mean(lemmas_vectors, axis=0)
        
    mult = vec.dot(rev_matrix)
    for_range = list(zip(mult, docs))
    sort_range = sorted(for_range, reverse=True)
        
    for indx, i in enumerate(sort_range):
        if indx < 5:
            if i[1][1] == '1':
                success.append(1)
                break
                    
        
accuracy = len(success) / len(quers)
print('Точность поиска = ', accuracy)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  del sys.path[0]


Точность поиска =  0.684


# Поиск с BM25

In [0]:
corp = corpus[:1000]

In [0]:
N = len(corp)

In [0]:
#длины документов
lens_d = []
for doc in corp:
    doc = doc.split()
    lens_d.append(len(doc))
lens_d[:5]

[7, 5, 5, 4, 8]

In [0]:
#средняя длина документа
avgdl = sum(lens_d)/len(lens_d)
avgdl

5.741

In [0]:
#константы
k = 2
b = 0.75

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
import numpy as np

In [0]:
X = vectorizer.fit_transform(corp)
#уникальные слова в корпусе
words = vectorizer.get_feature_names()

In [0]:
matrix = X.toarray()
#обратная матрица
rev_matrix = np.transpose(matrix)

In [0]:
#кол-во документов, где есть слово
number_of_docs = []
for word in rev_matrix:
    count = 0
    for i in word:
        if i != 0:
            count += 1
    number_of_docs.append(count)

In [0]:
TFs = []
for word in words:
    arr = []
    for sent in corp:
        TF = 0
        sent = sent.split()
        if word in sent:
            num = 0
            for s in sent:
                if word == s:
                    num += 1
            TF = num / len(sent)
        arr.append(TF)
    TFs.append(arr)

In [0]:
import math

In [0]:
#Создание матрицы с BM25
bm_mtrx = []
for index, word in enumerate(words):
    word_bms = []
    for indx, doc in enumerate(corp):
        l = float(lens_d[indx])
        n = float(number_of_docs[index])
        TF = float(TFs[index][indx])
        bm = math.log((N - n + 0.5) / (n + 0.5)) * (TF * (k + 1)) / (TF + (k * (1 - b + b * l / avgdl)))
        word_bms.append(bm)
    bm_mtrx.append(word_bms)

In [0]:
len(bm_mtrx)

2372

In [0]:
success = []
q_number = 0
quers = queries[:1000]

for q in quers:
    bms = []
    
    for i in q.split():
        for indx, word in enumerate(words):
            if i == word:
                bm = bm_mtrx[indx]
                bms.append(bm)
                
    if len(bms) > 0:
        q_bm = sum(np.array(bms))
        for_range = list(zip(q_bm, docs))
        sort_range = sorted(for_range, reverse=True)
        
        for indx, i in enumerate(sort_range):
            if indx < 5:
                if i[1][1] == '1':
                    success.append(1)
                    break
                    
    q_number += 1
        
accuracy = len(success) / q_number
print('Точность поиска = ', accuracy)

Точность поиска =  0.924


##### Итог: точность поиска с fasttext - 0.684; c BM25 - 0.924